# Encontro 05: Simulação de Centralidades

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

O objetivo desta atividade é realizar $24$ simulações de centralidade diferentes, para avaliar o desempenho de medidas clássicas em relação a diferentes processos de fluxo. Dessas $24$ simulações, são $12$ sobre o grafo `g1` e $12$ sobre o grafo `g2`.

In [3]:
g1 = sn.load_graph('renaissance.gml', has_pos=True)
g2 = sn.load_graph('../encontro02/1-introducao.gml', has_pos=True)

O primeiro grafo corresponde aos casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett, C. K. Ansell, 1993. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319.

In [4]:
sn.show_graph(g1, nlab=True)

O segundo grafo corresponde ao estudo de caso do primeiro encontro.

In [5]:
sn.show_graph(g2, nlab=True)

Das $12$ simulações sobre um dos grafos, são $6$ de *closeness* e $6$ de *betweenness*:

* duplicação serial através de caminhos;
* transferência através de caminhos;
* duplicação serial através de trilhas;
* transferência através de trilhas;
* duplicação serial através de passeios;
* transferência através de passeios.

In [6]:
from random import choice

TIMES = 1000

Em uma simulação de *closeness*, para cada origem `s` e destino `t`, mede-se o tempo que o fluxo leva para chegar de `s` a `t`. O *closeness simulado* de um nó `s` é a soma de todos os tempos medidos quando esse nó é uma origem. Como o fluxo pode ter passos aleatórios, o processo é repetido `TIMES` vezes e considera-se a média.

A função abaixo calcula o closeness simulado em relação a *transferência através de geodésicas*.

In [7]:
def simulate_closeness_transfer_geodesic(g):

    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                    m = choice(g.node[n]['shortest_neighbors'])
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

Vamos comparar o closeness simulado com o *closeness teórico*.

Para calcular o segundo, basta usar o *algoritmo de busca em largura* que vimos em encontros anteriores.

In [8]:
from math import inf, isinf
from queue import Queue

def build_closeness(g):
    for s in g.nodes():
        # início da busca em largura

        q = Queue()

        for n in g.nodes():
            g.node[n]['d'] = inf

        g.node[s]['d'] = 0
        q.put(s)

        while not q.empty():
            n = q.get()

            for m in g.neighbors(n):
                if isinf(g.node[m]['d']):

                    q.put(m)

        # fim da busca em largura

        g.node[s]['theoretical_closeness'] = 0

        for n in g.nodes():
            g.node[s]['theoretical_closeness'] += g.node[n]['d']

Comparação de closeness no grafo `g1`:

*(vai demorar alguns segundos)*

In [9]:
build_closeness(g1)

simulate_closeness_transfer_geodesic(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

ginori 42 42.0
lambertes 43 43.0
albizzi 29 29.0
guadagni 30 30.0
pazzi 49 49.0
salviati 36 36.0
medici 25 25.0
tornabuon 29 29.0
bischeri 35 35.0
ridolfi 29 29.0
acciaiuol 38 38.0
strozzi 33 33.0
peruzzi 40 40.0
barbadori 32 32.0
castellan 36 36.0


Comparação de closeness no grafo `g2`:

*(vai demorar alguns segundos)*

In [10]:
build_closeness(g2)

simulate_closeness_transfer_geodesic(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_closeness'], g2.node[n]['simulated_closeness'])

Rogerio 30 30.0
Roberto 36 36.0
Renato 47 47.0
Larissa 36 36.0
Jorge 49 49.0
Sueli 45 45.0
Conrado 30 30.0
Ricardo 38 38.0
Pamela 47 47.0
Fabio 49 49.0
Paulo 35 35.0
William 31 31.0
Tiago 38 38.0
Sandra 44 44.0
Patrick 41 41.0
Jose 40 40.0
Caio 46 46.0


Em uma simulação de *betweenness*, para cada origem `s`, destino `t` e intermediário `n`, mede-se a quantidade de vezes que o fluxo passa por `n` antes de chegar de `s` a `t`. O *betweenness simulado* de um nó `n` é a soma de todas as quantidades medidas quando esse nó é um intermediário. Como o fluxo pode ter passos aleatórios, o processo é repetido `TIMES` vezes e considera-se a média.

A função abaixo calcula o betweenness simulado em relação a *transferência através de geodésicas*.

In [11]:
def simulate_betweenness_transfer_geodesic(g):

    
    
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                    m = choice(g.node[n]['shortest_neighbors'])
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

Vamos comparar o betweenness simulado com o *betweennesss teórico*.

Para calcular o segundo, basta usar a função caixa-preta `build_betweenness`. Vocês vão aprender a abrir essa caixa-preta em encontros posteriores.

Comparação de betweenness no grafo `g1`:

*(vai demorar alguns segundos)*

In [12]:
sn.build_betweenness(g1)

simulate_betweenness_transfer_geodesic(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_betweenness'], g1.node[n]['simulated_betweenness'])

ginori 0.0 0.0
lambertes 0.0 0.0
albizzi 19.333333333333332 19.2025
guadagni 23.666666666666668 23.396
pazzi 0.0 0.0
salviati 13.0 13.0
medici 47.5 47.4765
tornabuon 8.333333333333336 8.1705
bischeri 11.0 11.005
ridolfi 7.833333333333333 8.115
acciaiuol 0.0 0.0
strozzi 6.833333333333333 7.0885
peruzzi 2.0 2.039
barbadori 10.5 10.4995
castellan 8.0 8.0075


Comparação de betweenness no grafo `g2`:

*(vai demorar alguns segundos)*

In [13]:
sn.build_betweenness(g2)

simulate_betweenness_transfer_geodesic(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_betweenness'], g2.node[n]['simulated_betweenness'])

Rogerio 33.416666666666664 33.3385
Roberto 20.333333333333336 20.3475
Renato 0.0 0.0
Larissa 15.083333333333334 15.103
Jorge 0.3333333333333333 0.336
Sueli 1.1666666666666665 1.154
Conrado 63.0 63.0
Ricardo 6.083333333333333 5.994
Pamela 0.0 0.0
Fabio 0.3333333333333333 0.349
Paulo 31.166666666666668 31.185
William 24.666666666666664 24.7455
Tiago 6.083333333333334 6.134
Sandra 2.9999999999999996 2.998
Patrick 0.0 0.0
Jose 0.0 0.0
Caio 0.3333333333333333 0.3155


## Entregáveis

Para **quinta 24/8**, você deve entregar todas as funções abaixo.

Funções auxiliares para evitar repetição de código são permitidas e encorajadas.

In [74]:
def simulate_closeness_serial_path(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
#                         for n in g.nodes():
#                             g.node[n]['owner'] = False
#                         g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # PATH: Os vizinhos válidos são os que ainda não passamos.
                                neigh = g.neighbors(n)
                                _n = []
                                for nn in neigh:
                                    if nn not in nodes_reached:
                                        _n.append(nn)
                                if len(_n) > 0:
                                    m = choice(_n)
                                else:
                                    break
                                nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                     g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
#                             for n in nodes_reached:
#                                 g.node[n]['owner'] = True

                            # Se alcançamos t, interroma\\\apemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

    pass

In [75]:
build_closeness(g1)

simulate_closeness_serial_path(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

ginori 42 52.554
lambertes 43 51.643
albizzi 29 53.491
guadagni 30 53.953
pazzi 49 53.935
salviati 36 52.654
medici 25 52.722
tornabuon 29 52.603
bischeri 35 51.428
ridolfi 29 53.241
acciaiuol 38 51.397
strozzi 33 52.44
peruzzi 40 51.275
barbadori 32 51.346
castellan 36 26.73


In [76]:
build_closeness(g2)

simulate_closeness_serial_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_closeness'], g2.node[n]['simulated_closeness'])

Rogerio 30 20.664
Roberto 36 20.692
Renato 47 20.111
Larissa 36 20.664
Jorge 49 20.35
Sueli 45 20.684
Conrado 30 20.245
Ricardo 38 20.473
Pamela 47 20.239
Fabio 49 20.234
Paulo 35 20.686
William 31 20.717
Tiago 38 20.551
Sandra 44 20.579
Patrick 41 19.988
Jose 40 20.312
Caio 46 20.486


In [79]:
def simulate_closeness_transfer_path(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # PATH: Os vizinhos válidos são os que ainda não passamos.
                                    neigh = g.neighbors(n)
                                    _n = []
                                    for nn in neigh:
                                        if nn not in nodes_reached:
                                            _n.append(nn)
                                    if len(_n) > 0:
                                        m = choice(_n)
                                    else:
                                        break
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interroma\\\apemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES


In [80]:
build_closeness(g1)

simulate_closeness_transfer_path(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

ginori 42 403.234
lambertes 43 413.592
albizzi 29 424.489
guadagni 30 432.264
pazzi 49 373.95
salviati 36 403.018
medici 25 434.389
tornabuon 29 434.709
bischeri 35 432.93
ridolfi 29 433.413
acciaiuol 38 408.279
strozzi 33 442.61
peruzzi 40 439.064
barbadori 32 424.668
castellan 36 430.781


In [81]:
build_closeness(g2)

simulate_closeness_transfer_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_closeness'], g2.node[n]['simulated_closeness'])

Rogerio 30 665.363
Roberto 36 630.989
Renato 47 626.793
Larissa 36 681.937
Jorge 49 665.826
Sueli 45 631.569
Conrado 30 647.148
Ricardo 38 676.076
Pamela 47 625.068
Fabio 49 661.418
Paulo 35 629.51
William 31 679.276
Tiago 38 682.844
Sandra 44 639.824
Patrick 41 672.339
Jose 40 689.12
Caio 46 630.429


In [82]:
def simulate_closeness_serial_trail(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
#                         for n in g.nodes():
#                             g.node[n]['owner'] = False
#                         g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()
                            edges_reached = set()
                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # TRAIL: Não repete aresta
                                e = g.edges(n)
                                ed = []
                                for ee in e:
                                    if ee not in edges_reached:
                                        ed.append(ee)
                                m = choice(ed)
                                edges_reached.add(m)
                                nodes_reached.add(m[0] if m[0] != n else m[1])
                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                     g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
#                             for n in nodes_reached:
#                                 g.node[n]['owner'] = True

                            # Se alcançamos t, interroma\\\apemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

    

In [83]:
build_closeness(g1)

simulate_closeness_serial_trail(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

ginori 42 27.862
lambertes 43 26.44
albizzi 29 29.122
guadagni 30 29.591
pazzi 49 28.441
salviati 36 29.623
medici 25 28.942
tornabuon 29 28.727
bischeri 35 28.906
ridolfi 29 28.766
acciaiuol 38 24.138
strozzi 33 28.815
peruzzi 40 28.5
barbadori 32 28.249
castellan 36 29.262


In [84]:
build_closeness(g2)

simulate_closeness_serial_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_closeness'], g2.node[n]['simulated_closeness'])

Rogerio 30 26.56
Roberto 36 26.336
Renato 47 26.072
Larissa 36 27.069
Jorge 49 26.182
Sueli 45 26.586
Conrado 30 26.096
Ricardo 38 26.639
Pamela 47 25.822
Fabio 49 25.923
Paulo 35 26.778
William 31 26.644
Tiago 38 26.899
Sandra 44 27.079
Patrick 41 25.175
Jose 40 25.906
Caio 46 26.452


In [85]:
def simulate_closeness_serial_walk(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # PATH: Os vizinhos válidos são os que ainda não passamos.
                                    neigh = g.neighbors(n)
                                    m = choice(neigh)
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                     g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interroma\\\apemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES


In [86]:
build_closeness(g1)

simulate_closeness_serial_walk(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

ginori 42 100.943
lambertes 43 98.762
albizzi 29 89.877
guadagni 30 89.422
pazzi 49 111.038
salviati 36 98.67
medici 25 84.661
tornabuon 29 84.585
bischeri 35 92.273
ridolfi 29 85.613
acciaiuol 38 93.033
strozzi 33 89.599
peruzzi 40 96.633
barbadori 32 86.729
castellan 36 94.02


In [87]:
build_closeness(g2)

simulate_closeness_serial_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_closeness'], g2.node[n]['simulated_closeness'])

Rogerio 30 103.315
Roberto 36 108.465
Renato 47 129.28
Larissa 36 116.453
Jorge 49 126.316
Sueli 45 124.939
Conrado 30 82.804
Ricardo 38 115.939
Pamela 47 126.027
Fabio 49 125.643
Paulo 35 111.976
William 31 101.822
Tiago 38 115.481
Sandra 44 125.906
Patrick 41 110.568
Jose 40 118.012
Caio 46 129.713


In [89]:
def simulate_closeness_transfer_walk(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # PATH: Os vizinhos válidos são os que ainda não passamos.
                                    neigh = g.neighbors(n)
                                    m = choice(neigh)
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interroma\\\apemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES


In [97]:
build_closeness(g1)

simulate_closeness_serial_walk(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

ginori 0.0 120.445
lambertes 0.0 81.8695
albizzi 19.333333333333332 252.7565
guadagni 23.666666666666668 252.546
pazzi 0.0 186.982
salviati 13.0 252.109
medici 47.5 252.2965
tornabuon 8.333333333333336 192.169
bischeri 11.0 83.3085
ridolfi 7.833333333333333 230.225
acciaiuol 0.0 65.636
strozzi 6.833333333333333 166.7325
peruzzi 2.0 92.4325
barbadori 10.5 65.3135
castellan 8.0 1.672


In [122]:
build_closeness(g2)

simulate_closeness_serial_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_closeness'], 
          g2.node[n]['simulated_closeness'])

Rogerio 30 103.285
Roberto 36 108.458
Renato 47 129.569
Larissa 36 116.781
Jorge 49 125.621
Sueli 45 124.144
Conrado 30 82.559
Ricardo 38 115.214
Pamela 47 125.946
Fabio 49 126.169
Paulo 35 111.626
William 31 101.54
Tiago 38 115.396
Sandra 44 125.782
Patrick 41 110.439
Jose 40 117.622
Caio 46 129.566


In [93]:
def simulate_betweenness_serial_path(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
#                         for n in g.nodes():
#                             g.node[n]['owner'] = False
#                         g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                neigh = g.neighbors(n)
                                _n = []
                                for nn in neigh:
                                    if nn not in nodes_reached:
                                        _n.append(nn)
                                if len(_n) > 0:
                                    m = choice(_n)
                                else:
                                    break
                                nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                 g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
#                             for n in nodes_reached:
#                                 g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [101]:
build_closeness(g1)

simulate_betweenness_serial_path(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

ginori 0.0 122.347
lambertes 0.0 83.2545
albizzi 19.333333333333332 256.7045
guadagni 23.666666666666668 256.584
pazzi 0.0 191.211
salviati 13.0 256.652
medici 47.5 256.563
tornabuon 8.333333333333336 195.3765
bischeri 11.0 84.9345
ridolfi 7.833333333333333 233.807
acciaiuol 0.0 66.5725
strozzi 6.833333333333333 169.3275
peruzzi 2.0 93.6165
barbadori 10.5 66.8775
castellan 8.0 1.6935


In [106]:
build_closeness(g2)

simulate_betweenness_serial_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

Rogerio 33.416666666666664 30.246
Roberto 20.333333333333336 31.8995
Renato 0.0 17.041
Larissa 15.083333333333334 33.334
Jorge 0.3333333333333333 25.273
Sueli 1.1666666666666665 30.8265
Conrado 63.0 20.425
Ricardo 6.083333333333333 26.7875
Pamela 0.0 20.9
Fabio 0.3333333333333333 22.1905
Paulo 31.166666666666668 28.6415
William 24.666666666666664 30.105
Tiago 6.083333333333334 29.348
Sandra 2.9999999999999996 31.3145
Patrick 0.0 15.7095
Jose 0.0 21.5085
Caio 0.3333333333333333 25.9665


In [103]:
def simulate_betweenness_transfer_path(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                neigh = g.neighbors(n)
                                _n = []
                                for nn in neigh:
                                    if nn not in nodes_reached:
                                        _n.append(nn)
                                if len(_n) > 0:
                                    m = choice(_n)
                                else:
                                    break
                                nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                 g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [104]:
build_closeness(g1)

simulate_betweenness_transfer_path(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

ginori 0.0 121.5945
lambertes 0.0 83.1575
albizzi 19.333333333333332 255.6135
guadagni 23.666666666666668 255.4985
pazzi 0.0 189.7405
salviati 13.0 255.3975
medici 47.5 255.0235
tornabuon 8.333333333333336 194.7
bischeri 11.0 84.5395
ridolfi 7.833333333333333 232.4685
acciaiuol 0.0 66.476
strozzi 6.833333333333333 168.1575
peruzzi 2.0 93.3305
barbadori 10.5 66.297
castellan 8.0 1.693


In [105]:
build_closeness(g2)

simulate_betweenness_transfer_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

Rogerio 33.416666666666664 30.0435
Roberto 20.333333333333336 31.7315
Renato 0.0 16.8675
Larissa 15.083333333333334 33.148
Jorge 0.3333333333333333 24.9945
Sueli 1.1666666666666665 30.7665
Conrado 63.0 20.3815
Ricardo 6.083333333333333 26.6015
Pamela 0.0 20.704
Fabio 0.3333333333333333 22.075
Paulo 31.166666666666668 28.608
William 24.666666666666664 29.872
Tiago 6.083333333333334 29.067
Sandra 2.9999999999999996 31.105
Patrick 0.0 15.5685
Jose 0.0 21.1205
Caio 0.3333333333333333 25.6855


In [109]:
def simulate_betweenness_serial_trail(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
#                         for n in g.nodes():
#                             g.node[n]['owner'] = False
#                         g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()
                            edges_reached = set()
                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                e = g.edges(n)
                                ed = []
                                for ee in e:
                                    if ee not in edges_reached:
                                        ed.append(ee)
                                m = choice(ed)
                                edges_reached.add(m)
                                nodes_reached.add(m[0] if m[0] != n else m[1])
                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                 g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
#                             for n in nodes_reached:
#                                 g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [110]:
build_closeness(g1)

simulate_betweenness_serial_trail(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

ginori 0.0 28.7645
lambertes 0.0 20.347
albizzi 19.333333333333332 99.1805
guadagni 23.666666666666668 99.244
pazzi 0.0 46.5085
salviati 13.0 99.2345
medici 47.5 99.162
tornabuon 8.333333333333336 57.262
bischeri 11.0 74.1175
ridolfi 7.833333333333333 60.8245
acciaiuol 0.0 11.771
strozzi 6.833333333333333 69.484
peruzzi 2.0 53.836
barbadori 10.5 42.548
castellan 8.0 82.2455


In [111]:
build_closeness(g2)

simulate_betweenness_serial_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

Rogerio 33.416666666666664 66.561
Roberto 20.333333333333336 58.9745
Renato 0.0 36.6625
Larissa 15.083333333333334 73.7835
Jorge 0.3333333333333333 39.8135
Sueli 1.1666666666666665 58.842
Conrado 63.0 41.308
Ricardo 6.083333333333333 60.452
Pamela 0.0 35.815
Fabio 0.3333333333333333 39.8145
Paulo 31.166666666666668 66.1175
William 24.666666666666664 60.0515
Tiago 6.083333333333334 60.558
Sandra 2.9999999999999996 67.7735
Patrick 0.0 27.608
Jose 0.0 37.025
Caio 0.3333333333333333 50.463


In [110]:
build_closeness(g1)

simulate_betweenness_serial_trail(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

ginori 0.0 28.7645
lambertes 0.0 20.347
albizzi 19.333333333333332 99.1805
guadagni 23.666666666666668 99.244
pazzi 0.0 46.5085
salviati 13.0 99.2345
medici 47.5 99.162
tornabuon 8.333333333333336 57.262
bischeri 11.0 74.1175
ridolfi 7.833333333333333 60.8245
acciaiuol 0.0 11.771
strozzi 6.833333333333333 69.484
peruzzi 2.0 53.836
barbadori 10.5 42.548
castellan 8.0 82.2455


In [111]:
build_closeness(g2)

simulate_betweenness_serial_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

Rogerio 33.416666666666664 66.561
Roberto 20.333333333333336 58.9745
Renato 0.0 36.6625
Larissa 15.083333333333334 73.7835
Jorge 0.3333333333333333 39.8135
Sueli 1.1666666666666665 58.842
Conrado 63.0 41.308
Ricardo 6.083333333333333 60.452
Pamela 0.0 35.815
Fabio 0.3333333333333333 39.8145
Paulo 31.166666666666668 66.1175
William 24.666666666666664 60.0515
Tiago 6.083333333333334 60.558
Sandra 2.9999999999999996 67.7735
Patrick 0.0 27.608
Jose 0.0 37.025
Caio 0.3333333333333333 50.463


In [112]:
def simulate_betweenness_transfer_trail(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()
                            edges_reached = set()
                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                e = g.edges(n)
                                ed = []
                                for ee in e:
                                    if ee not in edges_reached:
                                        ed.append(ee)
                                m = choice(ed)
                                edges_reached.add(m)
                                nodes_reached.add(m[0] if m[0] != n else m[1])
                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                 g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [113]:
build_closeness(g1)

simulate_betweenness_transfer_trail(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

ginori 0.0 28.6875
lambertes 0.0 20.5755
albizzi 19.333333333333332 99.7625
guadagni 23.666666666666668 100.021
pazzi 0.0 46.468
salviati 13.0 99.8645
medici 47.5 99.6495
tornabuon 8.333333333333336 57.55
bischeri 11.0 74.506
ridolfi 7.833333333333333 61.5085
acciaiuol 0.0 11.7465
strozzi 6.833333333333333 69.6615
peruzzi 2.0 54.3375
barbadori 10.5 42.699
castellan 8.0 82.466


In [114]:
build_closeness(g2)

simulate_betweenness_transfer_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

Rogerio 33.416666666666664 67.0105
Roberto 20.333333333333336 59.184
Renato 0.0 37.1475
Larissa 15.083333333333334 74.3225
Jorge 0.3333333333333333 40.2555
Sueli 1.1666666666666665 59.3375
Conrado 63.0 41.6875
Ricardo 6.083333333333333 61.052
Pamela 0.0 35.8205
Fabio 0.3333333333333333 40.0495
Paulo 31.166666666666668 66.6245
William 24.666666666666664 60.808
Tiago 6.083333333333334 61.09
Sandra 2.9999999999999996 68.409
Patrick 0.0 27.786
Jose 0.0 37.377
Caio 0.3333333333333333 50.882


In [116]:
def simulate_betweenness_serial_walk(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
#                         for n in g.nodes():
#                             g.node[n]['owner'] = False
#                         g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()
                            edges_reached = set()
                            for n in g.nodes():
#                                 if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                neigh = g.neighbors(n)
                                m = choice(neigh)
                                nodes_reached.add(m)
                                # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
#                                 g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
#                             for n in nodes_reached:
#                                 g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [120]:
build_closeness(g1)

simulate_betweenness_serial_walk(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

build_closeness(g2)

simulate_betweenness_serial_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

ginori 0.0 67.7895
lambertes 0.0 69.221
albizzi 19.333333333333332 223.487
guadagni 23.666666666666668 302.5055
pazzi 0.0 64.065
salviati 13.0 141.5385
medici 47.5 459.299
tornabuon 8.333333333333336 226.5965
bischeri 11.0 223.96
ridolfi 7.833333333333333 225.6385
acciaiuol 0.0 69.958
strozzi 6.833333333333333 224.2075
peruzzi 2.0 145.4925
barbadori 10.5 147.628
castellan 8.0 223.0075
Rogerio 33.416666666666664 431.075
Roberto 20.333333333333336 301.4695
Renato 0.0 173.981
Larissa 15.083333333333334 490.1295
Jorge 0.3333333333333333 177.5175
Sueli 1.1666666666666665 297.4725
Conrado 63.0 247.0955
Ricardo 6.083333333333333 365.316
Pamela 0.0 174.604
Fabio 0.3333333333333333 176.747
Paulo 31.166666666666668 361.7035
William 24.666666666666664 369.0025
Tiago 6.083333333333334 366.183
Sandra 2.9999999999999996 358.6915
Patrick 0.0 179.7135
Jose 0.0 240.212
Caio 0.3333333333333333 234.6325


In [118]:
def simulate_betweenness_transfer_walk(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()
                            
                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                    neigh = g.neighbors(n)
                                    m = choice(neigh)
                                    nodes_reached.add(m)
                                # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [121]:
build_closeness(g1)

simulate_betweenness_transfer_walk(g1)

for n in g1.nodes():
    print(g1.node[n]['label'],
          g1.node[n]['theoretical_betweenness'],
          g1.node[n]['simulated_betweenness'])

build_closeness(g2)

simulate_betweenness_transfer_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'],
          g2.node[n]['theoretical_betweenness'],
          g2.node[n]['simulated_betweenness'])

ginori 0.0 67.359
lambertes 0.0 68.728
albizzi 19.333333333333332 222.1495
guadagni 23.666666666666668 299.964
pazzi 0.0 63.736
salviati 13.0 140.566
medici 47.5 454.7915
tornabuon 8.333333333333336 224.359
bischeri 11.0 221.201
ridolfi 7.833333333333333 224.195
acciaiuol 0.0 69.3745
strozzi 6.833333333333333 222.0195
peruzzi 2.0 144.089
barbadori 10.5 146.115
castellan 8.0 220.0305
Rogerio 33.416666666666664 430.766
Roberto 20.333333333333336 301.9635
Renato 0.0 174.851
Larissa 15.083333333333334 489.973
Jorge 0.3333333333333333 177.211
Sueli 1.1666666666666665 296.8965
Conrado 63.0 247.7185
Ricardo 6.083333333333333 365.3345
Pamela 0.0 175.0165
Fabio 0.3333333333333333 177.862
Paulo 31.166666666666668 362.5505
William 24.666666666666664 368.6935
Tiago 6.083333333333334 365.782
Sandra 2.9999999999999996 358.3465
Patrick 0.0 178.9145
Jose 0.0 240.681
Caio 0.3333333333333333 235.6425


Para encontros posteriores, serão pedidas análises dos resultados dessas simulações.


## Avançado

Na consolidação do módulo de centralidade, você pode atingir conceito avançado no objetivo **traduzir conceitos sociológicos para teoria dos grafos e algoritmos em grafos** se formular e implementar uma proposta coerente de *degree simulado*. Essa proposta deve ser entregue na mesma data.